In [1]:
import pandas as pd
from eviltransform import wgs2gcj, gcj2wgs
import coordtransform

In [2]:
GS_data_set = {}
CS_data_set = {}

for year in range(2013,2026):
    GS_data_set[year] = pd.read_parquet(f"C:/Users/31155/Dropbox/EV-GasDualNetwork/Data/intermediate/yiwei/intermidiate/GS_cleaned/GS_data_cleaned_{year}.parquet")
    print(year," GS ",len(GS_data_set[year]))
for year in range(2015,2026):
    CS_data_set[year] = pd.read_parquet(f"C:/Users/31155/Dropbox/EV-GasDualNetwork/Data/intermediate/yiwei/intermidiate/CS_cleaned/CS_data_cleaned_{year}.parquet")
    print(year," CS ",len(CS_data_set[year]))

2013  GS  98458
2014  GS  102642
2015  GS  117307
2016  GS  118338
2017  GS  118193
2018  GS  104177
2019  GS  109464
2020  GS  113399
2021  GS  116279
2022  GS  105157
2023  GS  112000
2024  GS  113219
2025  GS  114421
2015  CS  1829
2016  CS  1870
2017  CS  4220
2018  CS  33175
2019  CS  56250
2020  CS  72149
2021  CS  93798
2022  CS  96181
2023  CS  118293
2024  CS  182254
2025  CS  265110


In [3]:
coordtransform.wgs84_to_gcj02(116.391, 39.907)

[116.39724095859891, 39.9084011088464]

In [4]:
coordtransform.gcj02_to_wgs84(116.391, 39.907)

[116.3847590414011, 39.905598891153595]

In [5]:
GS_data_set[2013]

,name,address,wgs84_x,wgs84_y,tel,pname,cityname,adname,大类,中类,小类,pname_EN,cityname_EN,corporation,paused
0,九环汽车液化加气站,曹家堰路３８,121.429478,31.215804,,上海市,上海市,长宁区,汽车服务,加气站,加气站,Shanghai,Shanghai,Other,0
1,强隆加油站,,121.345873,31.225219,,上海市,上海市,长宁区,汽车服务,加油站,中国石化,Shanghai,Shanghai,CPCC,0
2,中国海油加油站（哈密苑西南）,,121.361647,31.202550,,上海市,上海市,长宁区,汽车服务,加油站,加油站,Shanghai,Shanghai,Other,0
3,华江石油加油站（万航渡路）,万航渡路５０６,121.430557,31.226794,,上海市,上海市,长宁区,汽车服务,加油站,加油站,Shanghai,Shanghai,Other,0
4,剑河加油站,泉口路９２号,121.364185,31.209988,,上海市,上海市,长宁区,汽车服务,加油站,中国石化,Shanghai,Shanghai,CPCC,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101586,加油站（兴安康复医院西）,,124.106174,50.421509,,黑龙江省,大兴安岭地区,加格达奇,汽车服务,加油站,中国石油,Heilongjiang Province,Daxinganling area,CNPC,0
101587,加油站,,124.172164,50.417751,,黑龙江省,大兴安岭地区,加格达奇,汽车服务,加油站,中国石油,Heilongjiang Province,Daxinganling area,CNPC,0
101588,兴安石油加油站,,124.171191,50.418762,,黑龙江省,大兴安岭地区,加格达奇,汽车服务,加油站,加油站,Heilongjiang Province,Daxinganling area,Other,0
101589,加油站（华洋西一小区西北）,,124.116511,50.428458,,黑龙江省,大兴安岭地区,加格达奇,汽车服务,加油站,加油站,Heilongjiang Province,Daxinganling area,Other,0


In [14]:
# Vectorized approach for better performance
import numpy as np

def batch_convert_wgs84_to_gcj02(df):
    """
    Batch convert WGS-84 to GCJ-02 coordinates
    """
    df_result = df.copy()
    
    # Convert coordinates
    converted_coords = np.array([
        coordtransform.wgs84_to_gcj02(lon, lat) 
        for lon, lat in zip(df['wgs84_x'], df['wgs84_y'])
    ])
    
    df_result['wgs84_x'] = converted_coords[:, 0]
    df_result['wgs84_y'] = converted_coords[:, 1]
    
    return df_result

def batch_convert_gcj02_to_wgs84(df):
    """
    Batch convert GCJ-02 to WGS-84 coordinates
    """
    df_result = df.copy()
    
    # Convert coordinates
    converted_coords = np.array([
        coordtransform.gcj02_to_wgs84(lon, lat) 
        for lon, lat in zip(df['wgs84_x'], df['wgs84_y'])
    ])
    
    df_result['wgs84_x'] = converted_coords[:, 0]
    df_result['wgs84_y'] = converted_coords[:, 1]
    
    return df_result

def batch_convert_gcj02_to_wgs84_for_some_provinces(df, provinces):
    """
    Batch convert GCJ-02 to WGS-84 coordinates for specific provinces
    """
    df_result = df.copy()
    
    mask = df_result['pname'].isin(provinces)
    
    # Convert coordinates only for specified provinces
    converted_coords = np.array([
        coordtransform.gcj02_to_wgs84(lon, lat) 
        for lon, lat in zip(df_result.loc[mask, 'wgs84_x'], df_result.loc[mask, 'wgs84_y'])
    ])
    
    df_result.loc[mask, 'wgs84_x'] = converted_coords[:, 0]
    df_result.loc[mask, 'wgs84_y'] = converted_coords[:, 1]
    
    return df_result

In [10]:
def check_anchor_stations(df1, df2, match_columns = ['pname','cityname','adname','address','name','大类','中类','小类']):
    
    df1 = df1[df1['address'].str.strip().ne('') & df1['address'].str.strip().ne('[]')]
    df2 = df2[df2['address'].str.strip().ne('') & df2['address'].str.strip().ne('[]')]
    
    # Merge to find anchor stations
    merged = pd.merge(df1, df2, on=match_columns, suffixes=('_prev', '_next'))
    
    # City stats
    city_counts = merged['cityname'].value_counts().to_dict()
    print(f"Year {year}: matched anchor stations per city:", city_counts)
    
    # Distance calculation
    merged['shift_x'] = (merged['wgs84_x_next'] - merged['wgs84_x_prev']) * 100000
    merged['shift_y'] = (merged['wgs84_y_next'] - merged['wgs84_y_prev']) * 100000
    distance_stats = {
        'shift_x': merged['shift_x'].describe().to_dict(),
        'shift_y': merged['shift_y'].describe().to_dict()
    }
    
    agg_df = (
        merged.groupby(['pname', 'cityname'])
        .agg(
            count=('shift_x', 'size'),
            avg_shift_x=('shift_x', 'median'),
            avg_shift_y=('shift_y', 'median')
        )
        .reset_index()
    )
    return agg_df


In [13]:
check_anchor_stations(batch_convert_gcj02_to_wgs84(GS_data_set[2016]), GS_data_set[2017])

Year 2025: matched anchor stations per city: {'重庆市': 538, '保定市': 505, '石家庄市': 469, '临沂市': 446, '潍坊市': 439, '唐山市': 394, '北京市': 382, '烟台市': 371, '天津市': 371, '青岛市': 361, '南阳市': 351, '成都市': 348, '德州市': 327, '沧州市': 326, '济南市': 298, '淄博市': 287, '济宁市': 285, '上海市': 273, '杭州市': 270, '邯郸市': 268, '盐城市': 259, '哈尔滨市': 258, '郑州市': 253, '泉州市': 244, '广州市': 237, '菏泽市': 236, '沈阳市': 228, '榆林市': 228, '大连市': 224, '信阳市': 223, '长春市': 220, '聊城市': 218, '徐州市': 216, '南通市': 212, '邢台市': 207, '驻马店市': 205, '苏州市': 202, '宁波市': 200, '运城市': 199, '泰安市': 196, '临汾市': 196, '洛阳市': 193, '滨州市': 188, '新乡市': 187, '长沙市': 185, '鄂尔多斯市': 185, '焦作市': 180, '佛山市': 180, '张家口市': 178, '江门市': 177, '无锡市': 176, '平顶山市': 175, '威海市': 175, '周口市': 174, '惠州市': 173, '漳州市': 169, '廊坊市': 164, '枣庄市': 162, '开封市': 161, '渭南市': 158, '安阳市': 156, '武汉市': 156, '赤峰市': 146, '曲靖市': 143, '西安市': 143, '常州市': 142, '湛江市': 142, '宜昌市': 138, '襄阳市': 138, '赣州市': 137, '咸阳市': 136, '商丘市': 135, '昆明市': 135, '晋中市': 134, '吕梁市': 132, '日照市': 132, '梅州市': 132, '齐齐哈尔市': 131, '南宁市': 13

,pname,cityname,count,avg_shift_x,avg_shift_y
0,上海市,上海市,273,-1.421085e-09,-1.184794e-05
1,云南省,临沧市,28,-5.001987e-02,9.383854e-05
2,云南省,丽江市,24,-5.001200e-02,5.662107e-04
3,云南省,保山市,50,-5.001211e-02,9.983666e-02
4,云南省,大理白族自治州,58,-9.980299e-02,0.000000e+00
...,...,...,...,...,...
352,黑龙江省,绥化市,103,1.685123e-05,-9.987682e-02
353,黑龙江省,鸡西市,53,-6.969276e-01,-9.924352e-02
354,黑龙江省,鹤岗市,17,-3.323720e-04,1.001416e-01
355,黑龙江省,黑河市,50,5.001563e-02,-3.413202e-04


In [19]:
check_anchor_stations(GS_data_set[2024], batch_convert_gcj02_to_wgs84_for_some_provinces(GS_data_set[2025],['广东省','广西壮族自治区']))

Year 2025: matched anchor stations per city: {'重庆市': 1572, '保定市': 997, '石家庄市': 959, '临沂市': 893, '唐山市': 808, '南阳市': 805, '上海市': 735, '成都市': 699, '北京市': 685, '潍坊市': 684, '邢台市': 673, '沧州市': 671, '周口市': 647, '天津市': 616, '烟台市': 576, '济南市': 573, '邯郸市': 568, '菏泽市': 567, '驻马店市': 557, '商丘市': 555, '济宁市': 551, '哈尔滨市': 548, '德州市': 541, '青岛市': 539, '榆林市': 521, '洛阳市': 505, '信阳市': 504, '杭州市': 497, '苏州市': 491, '盐城市': 479, '运城市': 476, '昆明市': 475, '渭南市': 473, '新乡市': 461, '泉州市': 459, '常德市': 456, '赣州市': 450, '长沙市': 443, '聊城市': 440, '张家口市': 438, '遵义市': 438, '西安市': 430, '南通市': 427, '滨州市': 419, '临汾市': 418, '徐州市': 414, '平顶山市': 413, '廊坊市': 406, '武汉市': 396, '鄂尔多斯市': 390, '邵阳市': 390, '广州市': 389, '淄博市': 387, '安阳市': 387, '衡阳市': 385, '吕梁市': 384, '郴州市': 383, '长春市': 383, '衡水市': 383, '宁波市': 380, '黄冈市': 380, '无锡市': 374, '岳阳市': 372, '晋中市': 370, '襄阳市': 367, '泰安市': 363, '上饶市': 361, '江门市': 359, '曲靖市': 356, '宜昌市': 353, '承德市': 353, '咸阳市': 351, '宿州市': 349, '南宁市': 345, '绥化市': 344, '郑州市': 342, '毕节市': 341, '荆州市': 340, '忻州市': 339

,pname,cityname,count,avg_shift_x,avg_shift_y
0,上海市,上海市,735,-0.305445,-0.306645
1,云南省,临沧市,143,0.035045,0.083984
2,云南省,丽江市,116,0.071395,0.026129
3,云南省,保山市,170,-0.017282,-0.053684
4,云南省,大理白族自治州,246,-0.048038,0.093852
...,...,...,...,...,...
362,黑龙江省,绥化市,344,0.520704,0.402364
363,黑龙江省,鸡西市,119,0.436822,0.167655
364,黑龙江省,鹤岗市,80,0.011823,0.119240
365,黑龙江省,黑河市,127,-0.128998,-0.169618


In [16]:
GS_data_set[2021]['pname'].value_counts()

pname
山东省         11095
河北省          9798
河南省          9360
广东省          6155
江苏省          5680
湖南省          5477
四川省          5310
湖北省          4827
安徽省          4625
山西省          4377
云南省          4270
内蒙古自治区       4209
陕西省          4068
辽宁省          3954
浙江省          3761
广西壮族自治区      3203
江西省          3023
福建省          2959
黑龙江省         2930
贵州省          2855
新疆维吾尔自治区     2765
吉林省          2657
重庆市          1970
甘肃省          1685
北京市           995
天津市           916
上海市           863
宁夏回族自治区       810
青海省           794
海南省           530
西藏自治区         358
Name: count, dtype: int64